In [23]:
import os
from download import download
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

url = "http://vam.wfp.org/sites/data/WFPVAM_FoodPrices_05-12-2017.csv"
file_name = 'WFPVAM_FoodPrices_05-12-2017.csv'

def get_dataset(url, file_name):
    if not os.path.isfile(file_name):
        download(url, file_name)
    
    return pd.read_csv(file_name, encoding='latin-1')

df = get_dataset(url, file_name)

df.loc[df['um_name'] == 'MT']

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
53125,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,4,2012,2.110000e+06,http://www.fedearroz.com.co/new/precios.php
53126,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,5,2012,2.326667e+06,http://www.fedearroz.com.co/new/precios.php
53127,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,6,2012,2.520000e+06,http://www.fedearroz.com.co/new/precios.php
53128,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,7,2012,2.520000e+06,http://www.fedearroz.com.co/new/precios.php
53129,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,8,2012,2.520000e+06,http://www.fedearroz.com.co/new/precios.php
53130,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,9,2012,2.520000e+06,http://www.fedearroz.com.co/new/precios.php
53131,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,10,2012,2.396400e+06,http://www.fedearroz.com.co/new/precios.php
53132,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,11,2012,2.280000e+06,http://www.fedearroz.com.co/new/precios.php
53133,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,12,2012,2.280000e+06,http://www.fedearroz.com.co/new/precios.php
53134,57,Colombia,948,Cundinamarca,840,Bogota,133,Rice (white),67,COP,14,Wholesale,18,MT,1,2013,1.986000e+06,http://www.fedearroz.com.co/new/precios.php


In [8]:
#new = df.groupby("adm1_name")
#new.describe()
COUNTRY = 'adm0_name'
REGION = 'adm1_name'
CITY = 'mkt_name'
PROD = 'cm_name'
CURR = 'cur_name'
SELLER = 'pt_name'
UNIT = 'um_name'
MONTH = 'mp_month'
YEAR = 'mp_year'
DATE = 'date'
PRICE = 'mp_price'
SOURCE = 'mp_commoditysource'

dict = {}
for (product, unit), df_group in df.groupby(["cm_name", "um_name"]):
    if not product in dict:
        dict[product] = [unit]
    else:
        dict[product].append(unit)

dict2 = {key: value for key, value in dict.items() if len(value) > 1}


print(df.eval(UNIT).unique())



['KG' 'Day' 'Unit' 'L' 'USD/LCU' '30 pcs' '500 G' '125 G' 'Packet'
 '11.5 KG' 'Pound' 'Cuartilla' '90 KG' 'MT' 'Libra' 'Gallon' '380 G'
 'Month' '85 G' '500 ML' 'Course' '45 KG' 'Head' '100 KG' '50 KG' '91 KG'
 'Bunch' '100 Tubers' '650 G' '750 ML' '115 G' 'Marmite' '350 G' '385 G'
 '1.8 KG' '2 KG' '1.8 L' '300 G' '160 G' '400 G' '10 pcs' '150 G' '5 KG'
 '5 L' '200 G' '12.5 KG' 'Loaf' 'Dozen' 'Sack' '1.5 KG' '185 G' '10 KG'
 '750 G' '1.5 L' '12 KG' 'kWh' 'Package' 'Cubic meter' '18 KG' '60 KG'
 '25 KG' '3 L' '3 KG' '3.5 KG' '168 G']


In [22]:
COUNTRY = 'adm0_name'
REGION = 'adm1_name'
CITY = 'mkt_name'
PROD = 'cm_name'
CURR = 'cur_name'
SELLER = 'pt_name'
UNIT = 'um_name'
MONTH = 'mp_month'
YEAR = 'mp_year'
DATE = 'date'
PRICE = 'mp_price'
SOURCE = 'mp_commoditysource'

print(len(df.eval(UNIT).unique()))

# assuming:
# 1 Libra = 0.329 kg
UNIT_PRICE_CONVERTER = {'1.5 KG': (1.5, 'KG'), '5 KG': (5.0, 'KG'), '1.8 KG': (1.8, 'KG'), 
                        '2 KG': (2.0, 'KG'), '12.5 KG': (12.5, 'KG'), '10 KG': (10.0, 'KG'),
                        '60 KG': (60.0, 'KG'), '18 KG': (18.0, 'KG'), '25 KG': (25.0, 'KG'),
                        '3 KG': (3.0, 'KG'), '3.5 KG': (3.5, 'KG'), '12 KG': (12.0, 'KG'),
                        '500 G': (0.5, 'KG'), '125 G': (0.125, 'KG'), '90 KG': (90.0, 'KG'), 
                        'Pound': (0.45359237, 'KG'), '380 G': (0.380, 'KG'), '85 G': (0.085, 'KG'),
                        '45 KG': (45.0, 'KG'), '100 KG': (100.0, 'KG'), '50 KG': (50.0, 'KG'), 
                        '91 KG': (91.0, 'KG'), '650 G': (0.650, 'KG'), '115 G': (0.115, 'KG'),
                        '350 G': (0.350, 'KG'), '385 G': (0.385, 'KG'), '11.5 KG': (11.5, 'KG'),
                        '400 G': (0.400, 'KG'), '150 G': (0.150, 'KG'), '160 G': (0.160, 'KG'), 
                        '200 G': (0.200, 'KG'), '185 G': (0.185, 'KG'), '750 G': (0.750, 'KG'),
                        '168 G': (0.168, 'KG'), 'Libra': (0.329, 'KG'),
                        'Gallon': (3.78541178, 'L'), '500 ML': (0.500, 'L'), '750 ML': (0.750, 'L'),
                        '1.5 L': (1.5, 'L'), '3 L': (3.0, 'L'), 'Cubic Meter': (1000.0, 'L'),
                        '5 L': (5.0, 'L'), 
                        '10 pcs': (10.0, 'pcs'), '30 pcs': (30.0, 'pcs'), 'Dozen': (12.0, "pcs")}
                        
                        

CONV_PRICE = 0
CONV_UNIT = 1

def get_values_column(df, column_name, value):
    """
    df = dataset
    column_name = kolom waarin je wil zoeken
    value = waarde die je uit de kolom wil.

    return nieuwe dataset met alle rijen die alleen de value in de gegeven kolommen bevatten.
    """
    return df.loc[df[column_name] == value]

def norm_price_unit(row, Col):
    """

    """
    if Col:
        try:
            return UNIT_PRICE_CONVERTER[row.get(UNIT)][Col]
        except:
            return row.get(UNIT)
    else:
        try:
            return row.get(PRICE) / UNIT_PRICE_CONVERTER[row.get(UNIT)][Col]
        except:
            return row.get(PRICE)

if __name__ == "__main__":
    df = pd.read_csv('WFPVAM_FoodPrices_version1.csv')
    # unique_per_cat(df)
    # print(df.size)
    
    
    #print(get_values_column(df, UNIT, '1.5 KG'))
    df[PRICE] = df.apply(lambda row: norm_price_unit(row, CONV_PRICE), axis = 1)
    df[UNIT] = df.apply(lambda row: norm_price_unit(row, CONV_UNIT), axis = 1)

    #print(get_values_column(df, UNIT, '1.5 KG'))

print(len(df.eval(UNIT).unique()))
print(df.eval(UNIT).unique())




#print(df["um_name"].unique())
#df["mp_price"] = df.apply(lambda row: row.get("mp_price") / 1.5 
                         # if row.get("um_name") == "KG" 
                          # else row.get("mp_price"), axis = 1)
#df.loc[df['um_name'] == "KG", 'um_name'] = "new unit"
#df[:5]


# copy = df.copy()
# original_prices = df.loc[df['um_name'] == 'KG', ['mp_price']]
# normalized_prices = original_prices / 2
# print(normalized_prices[:5])
#df['mp_price'] = df["mp_price"].replace(original_prices, normalized_prices)
#df['mp_price']
#df.loc[df['um_name'] == 'KG', ['um_name', 'mp_price']] = ['new unit', normalized_prices]


65
18
['KG' 'Day' 'Unit' 'L' 'USD/LCU' 'Cuartilla' 'MT' 'Month' 'Course' 'Head'
 'Bunch' '100 Tubers' 'Marmite' 'pcs' 'Loaf' 'Sack' 'Packet' 'Cubic meter']
